# Acoplador dos dados de *.las com o respectivo *.agp

## Importando módulos externos

In [1]:
######################PACOTES##########################
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import logging
sys.path.insert(0,'../modulos')
from graficador import plotagem as plm2
#######################################################

## Leitura dos dados de perfilagem

In [2]:
#----------------------------------------------------------------------#
# Leitura do data frame que contém os canais da perfilagem             #
#----------------------------------------------------------------------#
df = pd.read_csv("../saidas/1-ESS-120_resample_COT.txt", sep='\s+', skiprows=1 , 
                 names=('Depth(m)','COT','DT','GR','NPHI','RHOB'))

### Vetores perfilagem

In [3]:
depth=np.array(df['Depth(m)'])
COT=np.array(df['COT'])
DT=np.array(df['DT'])
GR=np.array(df['GR'])
NPHI=np.array(df['NPHI'])
RHOB=np.array(df['RHOB'])
#Cria um vetor numérico e um vetor de characters
code = np.zeros(len((df['Depth(m)'])))
rock = [0.0] * np.size(code, axis=0)#Esta variável precisa ser um character

In [4]:
df

,Depth(m),COT,DT,GR,NPHI,RHOB
0,4800.1046,5.399496,70.30501,157.6507,14.05369,2.616479
1,4800.3046,4.321568,73.25121,127.1674,11.76468,2.605533
2,4800.5046,4.060281,76.35267,119.7623,17.90119,2.585634
3,4800.7046,4.223565,78.27454,124.3640,21.43119,2.587109
4,4800.9046,4.049650,76.25685,119.4625,19.73067,2.608446
...,...,...,...,...,...,...
7335,6395.9046,5.415883,74.54109,158.4027,12.76359,2.693078
7336,6396.1046,5.415855,73.37498,158.4027,12.04690,2.707989
7337,6396.3046,5.416453,72.57875,158.4027,11.25337,2.676428
7338,6396.5046,5.416614,72.42371,158.4027,11.04827,2.661956


## Leitura do arquivo *.agp

In [5]:
#---------------------------------------------------------#
# Leitura do arquivo *.apg e criação do DataFrame:        #
#---------------------------------------------------------#

lito = pd.read_csv("../entradas/agpmod/1-ESS-120-ES.txt", sep='\s+', usecols=(0,1,2), 
                   index_col=False, na_values= ' ', skiprows=1, names=('TOPO(m)', 'BASE(m)','Rock') ) 

In [6]:

lito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458 entries, 0 to 457
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   TOPO(m)  458 non-null    float64
 1   BASE(m)  458 non-null    float64
 2   Rock     458 non-null    object 
dtypes: float64(2), object(1)
memory usage: 10.9+ KB


In [7]:
print(lito)

     TOPO(m)  BASE(m)         Rock
0     2026.0   2525.0     FOLHELHO
1     2525.0   2768.0      ARENITO
2     2768.0   2773.0        MARGA
3     2773.0   2814.5     FOLHELHO
4     2814.5   2815.5  CALCILUTITO
..       ...      ...          ...
453   6301.0   6306.0        MARGA
454   6306.0   6307.5        AREIA
455   6307.5   6310.0     FOLHELHO
456   6310.0   6314.0        AREIA
457   6314.0   6409.0     FOLHELHO

[458 rows x 3 columns]


### Vetores agp

In [8]:
topo=np.array(lito['TOPO(m)'])
base=np.array(lito['BASE(m)'])
rock_apg=np.array(lito['Rock'])

## Leitura do arquivo de codificação

In [9]:
codigos =  pd.read_csv("../entradas/colors.txt", sep=',', usecols=(0,1,2,3), 
                   index_col=False, na_values= ' ', skiprows=1, names=('Litologia', 'Abreviacao', 'Cor', 'Codigo') ) 

In [10]:
codigos

,Litologia,Abreviacao,Cor,Codigo
0,ANIDRITA,AND,#dd1dff,82
1,ARDOSIA,ARS,#00dd00,77
2,AREIA,ARE,#ffff3f,48
3,ARENITO,ARN,#ffff3f,49
4,ARENITO ARGILOSO,ARL,#7eff00,25
...,...,...,...,...
72,TUFO VULCANICO,TUF,#ff5dbe,61
73,ULTRABASICA,ULB,#00a000,78
74,VULCANICA NAO IDENTIFICADA,VNI,#ffbe1d,34
75,WACKESTONE,WKS,#33d8f9,13


### Vetores de codigos e cores (litologias base)

In [11]:
ROCKbase=np.array(codigos['Litologia'])
hexbase=np.array(codigos['Cor'])
codbase=np.array(codigos['Codigo'])

# Acoplamento

In [12]:
lito['Codigo']=np.zeros(len((lito['Rock'])))
lito

,TOPO(m),BASE(m),Rock,Codigo
0,2026.0,2525.0,FOLHELHO,0.0
1,2525.0,2768.0,ARENITO,0.0
2,2768.0,2773.0,MARGA,0.0
3,2773.0,2814.5,FOLHELHO,0.0
4,2814.5,2815.5,CALCILUTITO,0.0
...,...,...,...,...
453,6301.0,6306.0,MARGA,0.0
454,6306.0,6307.5,AREIA,0.0
455,6307.5,6310.0,FOLHELHO,0.0
456,6310.0,6314.0,AREIA,0.0


In [13]:
#Cria coluna de códigos no AGP
for i in range(len(lito)):
    for j in range(len(codigos)):
        if lito['Rock'][i] == codigos['Litologia'][j]:
            lito['Codigo'][i]=codigos['Codigo'][j]

/tmp/ipykernel_12801/3844772693.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lito['Codigo'][i]=codigos['Codigo'][j]


In [14]:
print(lito)

     TOPO(m)  BASE(m)         Rock  Codigo
0     2026.0   2525.0     FOLHELHO    57.0
1     2525.0   2768.0      ARENITO    49.0
2     2768.0   2773.0        MARGA    58.0
3     2773.0   2814.5     FOLHELHO    57.0
4     2814.5   2815.5  CALCILUTITO     6.0
..       ...      ...          ...     ...
453   6301.0   6306.0        MARGA    58.0
454   6306.0   6307.5        AREIA    48.0
455   6307.5   6310.0     FOLHELHO    57.0
456   6310.0   6314.0        AREIA    48.0
457   6314.0   6409.0     FOLHELHO    57.0

[458 rows x 4 columns]


In [15]:
codagp=np.array(lito['Codigo'])

In [16]:
print(code)

[0. 0. 0. ... 0. 0. 0.]


In [17]:
print(depth)
print(base)

[4800.1046 4800.3046 4800.5046 ... 6396.3046 6396.5046 6396.7046]
[2525.  2768.  2773.  2814.5 2815.5 2816.5 2817.5 2820.  2851.  2853.5
 2855.5 2859.  2865.  2867.  2870.5 2872.  2880.  2882.5 2885.5 2886.5
 2890.  2965.  2970.  2989.  2990.5 3011.5 3014.5 3031.5 3034.  3038.
 3045.  3046.5 3048.  3049.5 3055.5 3057.  3064.  3075.  3077.  3078.
 3087.  3091.  3094.  3095.  3101.  3104.5 3109.  3111.  3113.  3115.
 3118.5 3119.5 3121.5 3126.  3126.5 3127.5 3128.5 3130.  3131.5 3133.
 3134.  3135.5 3136.5 3138.5 3145.  3146.  3152.  3153.  3155.  3156.
 3160.  3162.  3166.  3167.  3168.  3170.  3171.  3172.  3173.5 3175.5
 3178.  3179.  3185.  3190.  3192.  3198.  3218.  3220.  3236.5 3238.5
 3248.5 3250.5 3265.  3267.  3273.  3274.  3290.5 3291.5 3299.  3300.
 3308.5 3312.  4179.  4181.  4422.  4424.  4425.  4428.  4557.5 4558.5
 4574.  4575.  4661.5 4662.  4663.  4663.5 4671.5 4672.5 4711.5 4712.
 4718.  4718.5 4720.5 4721.5 4722.  4726.5 4727.5 4750.  4750.5 4777.5
 4779.5 4792.5 479

In [18]:
#Limites dos laços:
fim1 = np.size(df['Depth(m)'], axis=0)# amostragem 0.2
fim2 = np.size(lito['BASE(m)'], axis=0)# agp

print(fim1,fim2)

7340 458


In [19]:
#--------------------------------------------------------#
#   Armazenando as informações nas variáveis             #
# Esta info é especifica para cada caso de poço          #
#--------------------------------------------------------#

for j in range(fim1-1):#poço
    for i in range(1,fim2):#agp
        if base[j] > depth[i-1] and base[j] <= depth[i]: 
            code[j]=codagp[i]
            #rock[j] = rockbase[i]
            print(j,i, base[j], depth[i-1],depth[i],code[j],codagp[i])
    
        # condicao extrema, ou seja, quando as profs dos perfis forem maiores que as do arquivo agp:
        if base[j] > depth[fim2-1]:
            code[j] = codagp[fim2-1]
            #rock[j] = rockbase[fim2-1] 

133 30 4806.0 4805.9046 4806.104600000001 57.0 57.0
134 40 4808.0 4807.9046 4808.104600000001 49.0 49.0
135 45 4809.0 4808.9046 4809.104600000001 57.0 57.0
136 60 4812.0 4811.9046 4812.104600000001 57.0 57.0
137 95 4819.0 4818.9046 4819.104600000001 57.0 57.0
138 105 4821.0 4820.9046 4821.104600000001 48.0 48.0
139 115 4823.0 4822.9046 4823.104600000001 58.0 58.0
140 120 4824.0 4823.9046 4824.104600000001 57.0 57.0
141 125 4825.0 4824.9046 4825.104600000001 57.0 57.0
142 127 4825.5 4825.3046 4825.5046 57.0 57.0
143 132 4826.5 4826.3046 4826.5046 6.0 6.0
144 140 4828.0 4827.9046 4828.104600000001 48.0 48.0
145 205 4841.0 4840.9046 4841.104600000001 48.0 48.0
146 215 4843.0 4842.9046 4843.104600000001 48.0 48.0
147 227 4845.5 4845.3046 4845.5046 48.0 48.0
148 260 4852.0 4851.9046 4852.104600000001 48.0 48.0
149 287 4857.5 4857.3046 4857.5046 57.0 57.0
150 292 4858.5 4858.3046 4858.5046 6.0 6.0
151 327 4865.5 4865.3046 4865.5046 57.0 57.0
152 332 4866.5 4866.3046 4866.5046 57.0 57.0
153 3

IndexError: index 458 is out of bounds for axis 0 with size 458